In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Load the normal data
df_normal = pd.read_csv('normall.csv')

# Extract feature columns
X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

# Standardize the data (important for DBSCAN)
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

# Function to perform grid search to find optimal DBSCAN parameters
def manual_grid_search_dbscan(data, eps_values, min_samples_values):
    best_score = -1
    best_params = {'eps': None, 'min_samples': None}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(data)
            # Ignore if all points are considered noise
            if len(set(labels)) > 1:
                score = silhouette_score(data, labels)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}
    return best_params

# Define the range of eps and min_samples values
eps_values = np.arange(0.5, 2.1, 0.1)  # Adjusted range
min_samples_values = np.arange(3, 10)

# Find optimal parameters
best_params = manual_grid_search_dbscan(X_normal_scaled, eps_values, min_samples_values)
eps = best_params['eps']
min_samples = best_params['min_samples']

print(f"Optimal eps: {eps}, Optimal min_samples: {min_samples}")

# Train the DBSCAN model with optimal parameters
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_normal_scaled)

# Filter out anomalies from the normal data
normal_data_filtered = X_normal_scaled[labels >= 0]

# Re-train DBSCAN on the filtered normal data
dbscan_filtered = DBSCAN(eps=eps, min_samples=min_samples)
dbscan_filtered.fit(normal_data_filtered)

# Ensure there are no anomalies in the filtered normal training data
assert np.all(dbscan_filtered.labels_ >= 0), "There should be no anomalies in the filtered normal training data"

print("Cluster labels for filtered normal data:", dbscan_filtered.labels_)

# Function to detect anomalies
def is_anomaly(sample, dbscan_model, scaler):
    sample_scaled = scaler.transform([sample])
    labels = dbscan_model.fit_predict(np.vstack([dbscan_model.components_, sample_scaled]))
    return labels[-1] == -1  # If the label is -1, it's an anomaly

# Example test samples (assumed normal and anomalous samples)
new_sample_normal = [-0.469915051151626, 0.275878711735454,-0.0275614966845328,-0.0887079422926568]  # Example normal data
new_sample_anomaly = [0.0620035945873911,-0.110699903969443,-0.0376392274991288,0.0168727730326617]  # Example anomalous data

# Check if the samples are anomalies
print("Is the normal sample an anomaly?", is_anomaly(new_sample_normal, dbscan_filtered, scaler))
print("Is the anomalous sample an anomaly?", is_anomaly(new_sample_anomaly, dbscan_filtered, scaler))


Optimal eps: 1.2999999999999998, Optimal min_samples: 3
Cluster labels for filtered normal data: [0 0 0 ... 0 0 0]
Is the normal sample an anomaly? True
Is the anomalous sample an anomaly? False


In [6]:
#DBSCAN (Density-Based Spatial Clustering of Applications with Noise)
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, accuracy_score

df_normal = pd.read_csv('normall.csv')
print(df_normal.shape)

X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

def manual_grid_search_dbscan(data, eps_values, min_samples_values):
    best_score = -1
    best_params = {'eps': None, 'min_samples': None}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(data)
            if len(set(labels)) > 1:
                score = silhouette_score(data, labels)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}
    return best_params

eps_values = np.arange(0.5, 2.1, 0.1)
min_samples_values = np.arange(3, 10)

# Find optimal parameters
best_params = manual_grid_search_dbscan(X_normal_scaled, eps_values, min_samples_values)
eps = best_params['eps']
min_samples = best_params['min_samples']

print(f"Optimal eps: {eps}, Optimal min_samples: {min_samples}")

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_normal_scaled)

normal_data_filtered = X_normal_scaled[labels >= 0]

dbscan_filtered = DBSCAN(eps=eps, min_samples=min_samples)
dbscan_filtered.fit(normal_data_filtered)

assert np.all(dbscan_filtered.labels_ >= 0), "There should be no anomalies in the filtered normal training data"

print("Cluster labels for filtered normal data:", dbscan_filtered.labels_)

def is_anomaly(sample, dbscan_model, scaler):
    sample_scaled = scaler.transform([sample])
    labels = dbscan_model.fit_predict(np.vstack([dbscan_model.components_, sample_scaled]))
    return labels[-1] == -1 

df_anomaly = pd.read_csv('anomlaydetection.csv')
print(df_anomaly.shape)

X_anomaly = df_anomaly[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values
y_true = df_anomaly['Anomaly'].values

X_anomaly_scaled = scaler.transform(X_anomaly)

y_pred = [is_anomaly(sample, dbscan_filtered, scaler) for sample in X_anomaly]

y_pred = np.array(y_pred, dtype=int)

y_true = np.array(y_true, dtype=int)

accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy of the model on the anomaly detection dataset: {accuracy:.2f}")


(4999, 4)
Optimal eps: 1.2999999999999998, Optimal min_samples: 3
Cluster labels for filtered normal data: [0 0 0 ... 0 0 0]
(595, 5)
Accuracy of the model on the anomaly detection dataset: 1.00


In [4]:
df = pd.read_csv('normall.csv')
df.shape

(4999, 4)

In [5]:
df = pd.read_csv('anomlaydetection.csv')
df.shape

(595, 5)

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Load the normal data
df_normal = pd.read_csv('normall.csv')

# Extract feature columns
X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

# Standardize the data (important for DBSCAN)
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

# Function to perform grid search to find optimal DBSCAN parameters
def manual_grid_search_dbscan(data, eps_values, min_samples_values):
    best_score = -1
    best_params = {'eps': None, 'min_samples': None}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(data)
            # Ignore if all points are considered noise
            if len(set(labels)) > 1:
                score = silhouette_score(data, labels)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}
    return best_params

# Define the range of eps and min_samples values
eps_values = np.arange(0.5, 2.1, 0.1)  # Adjusted range
min_samples_values = np.arange(3, 10)

# Find optimal parameters
best_params = manual_grid_search_dbscan(X_normal_scaled, eps_values, min_samples_values)
eps = best_params['eps']
min_samples = best_params['min_samples']

print(f"Optimal eps: {eps}, Optimal min_samples: {min_samples}")

# Train the DBSCAN model with optimal parameters
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_normal_scaled)

# Filter out anomalies from the normal data
normal_data_filtered = X_normal_scaled[labels >= 0]

# Re-train DBSCAN on the filtered normal data
dbscan_filtered = DBSCAN(eps=eps, min_samples=min_samples)
dbscan_filtered.fit(normal_data_filtered)

# Ensure there are no anomalies in the filtered normal training data
assert np.all(dbscan_filtered.labels_ >= 0), "There should be no anomalies in the filtered normal training data"

print("Cluster labels for filtered normal data:", dbscan_filtered.labels_)

# Function to detect anomalies
def is_anomaly(sample, dbscan_model, scaler):
    sample_scaled = scaler.transform([sample])
    labels = dbscan_model.fit_predict(np.vstack([dbscan_model.components_, sample_scaled]))
    return labels[-1] == -1  # If the label is -1, it's an anomaly

# Example test samples (assumed normal and anomalous samples)
new_sample_normal = [0.189630843,0.012816744,-0.022304104,0.020324712]  # Example normal data
new_sample_anomaly = [0.0620035945873911,-0.110699903969443,-0.0376392274991288,0.0168727730326617]  # Example anomalous data

# Check if the samples are anomalies
print("Is the normal sample an anomaly?", is_anomaly(new_sample_normal, dbscan_filtered, scaler))
print("Is the anomalous sample an anomaly?", is_anomaly(new_sample_anomaly, dbscan_filtered, scaler))


Optimal eps: 1.2999999999999998, Optimal min_samples: 3
Cluster labels for filtered normal data: [0 0 0 ... 0 0 0]
Is the normal sample an anomaly? False
Is the anomalous sample an anomaly? False


In [1]:
#svm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

# Load the normal data
df_normal = pd.read_csv('normall.csv')
print(df_normal.shape)

# Extract feature columns
X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

# Standardize the data
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

# Train the One-Class SVM model
svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # you can tune these hyperparameters
svm.fit(X_normal_scaled)

# Load the anomaly detection data
df_anomaly = pd.read_csv('anomlaydetection.csv')
print(df_anomaly.shape)

# Extract features and true labels
X_anomaly = df_anomaly[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values
y_true = df_anomaly['Anomaly'].values

# Standardize the anomaly detection data
X_anomaly_scaled = scaler.transform(X_anomaly)

# Predict anomalies using the trained One-Class SVM
y_pred = svm.predict(X_anomaly_scaled)

# One-Class SVM outputs -1 for anomalies and 1 for normal points
# Convert predictions to binary format (0 for normal, 1 for anomaly)
y_pred = np.where(y_pred == -1, 1, 0)

# Convert true labels to integers (0 for normal, 1 for anomaly)
y_true = np.array(y_true, dtype=int)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy of the One-Class SVM model on the anomaly detection dataset: {accuracy:.2f}")


(4999, 4)
(595, 5)
Accuracy of the One-Class SVM model on the anomaly detection dataset: 0.99


In [4]:
#ensemble
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the normal data
df_normal = pd.read_csv('normall.csv')
print(df_normal.shape)

# Extract feature columns
X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

# Standardize the data
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)  # Reduce to 2 dimensions
X_normal_pca = pca.fit_transform(X_normal_scaled)

# Use GridSearchCV to find the best bandwidth parameter for KDE
params = {'bandwidth': np.linspace(0.1, 2.0, 30)}
grid_search = GridSearchCV(KernelDensity(kernel='gaussian'), params, cv=5)
grid_search.fit(X_normal_pca)

# Get the best bandwidth
best_bandwidth = grid_search.best_estimator_.bandwidth
print(f"Optimal bandwidth: {best_bandwidth}")

# Train the Kernel Density model with the optimal bandwidth
kde = KernelDensity(kernel='gaussian', bandwidth=best_bandwidth)
kde.fit(X_normal_pca)

# Train the Isolation Forest model
iso_forest = IsolationForest(contamination=0.05)  # Adjust contamination as needed
iso_forest.fit(X_normal_pca)

# Load the anomaly detection data
df_anomaly = pd.read_csv('anomlaydetection.csv')
print(df_anomaly.shape)

# Extract features and true labels
X_anomaly = df_anomaly[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values
y_true = df_anomaly['Anomaly'].values

# Standardize the anomaly detection data
X_anomaly_scaled = scaler.transform(X_anomaly)

# Apply PCA transformation
X_anomaly_pca = pca.transform(X_anomaly_scaled)

# Predict anomalies using the trained Kernel Density model
log_density = kde.score_samples(X_anomaly_pca)

# Predict anomalies using the trained Isolation Forest model
iso_pred = iso_forest.predict(X_anomaly_pca)

# Define a threshold for KDE-based anomaly detection
kde_threshold = np.percentile(log_density, 5)  # Bottom 5% density
kde_pred = log_density < kde_threshold

# Combine predictions (using OR logic: if either model predicts anomaly, it is an anomaly)
y_pred = np.logical_or(kde_pred, iso_pred == -1).astype(int)

# Convert true labels to integers (0 for normal, 1 for anomaly)
y_true = np.array(y_true, dtype=int)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy of the combined model on the anomaly detection dataset: {accuracy:.2f}")



(4999, 4)
Optimal bandwidth: 0.4275862068965517
(595, 5)
Accuracy of the combined model on the anomaly detection dataset: 0.98


In [5]:
#KernelDensity
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score

# Load the normal data
df_normal = pd.read_csv('normall.csv')
print(df_normal.shape)

# Extract feature columns
X_normal = df_normal[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values

# Standardize the data
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)

# Load the anomaly detection data
df_anomaly = pd.read_csv('anomlaydetection.csv')
print(df_anomaly.shape)

# Extract features and true labels
X_anomaly = df_anomaly[['bearingA_x', 'bearingA_y', 'bearingB_x', 'bearingB_y']].values
y_true = df_anomaly['Anomaly'].values

# Standardize the anomaly detection data
X_anomaly_scaled = scaler.transform(X_anomaly)

# Parameters to try
bandwidths = np.linspace(0.1, 2.0, 20)
kernels = ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']

# Initialize variables to store the best results
best_accuracy = 0
best_params = {'bandwidth': None, 'kernel': None}

# Grid search over bandwidths and kernels
for bandwidth in bandwidths:
    for kernel in kernels:
        # Train the Kernel Density model
        kde = KernelDensity(kernel=kernel, bandwidth=bandwidth)
        kde.fit(X_normal_scaled)
        
        # Predict anomalies using the trained Kernel Density model
        log_density = kde.score_samples(X_anomaly_scaled)
        
        # Define a threshold for anomaly detection
        threshold = np.percentile(log_density, 5)  # for example, the bottom 5% density
        y_pred = log_density < threshold
        
        # Convert boolean predictions to integers (0 for normal, 1 for anomaly)
        y_pred = np.array(y_pred, dtype=int)
        
        # Convert true labels to integers (0 for normal, 1 for anomaly)
        y_true = np.array(y_true, dtype=int)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_true, y_pred)
        
        # Update best parameters if current accuracy is better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['bandwidth'] = bandwidth
            best_params['kernel'] = kernel

        print(f"Bandwidth: {bandwidth}, Kernel: {kernel}, Accuracy: {accuracy:.2f}")

print(f"Best Accuracy: {best_accuracy:.2f} with Bandwidth: {best_params['bandwidth']} and Kernel: {best_params['kernel']}")


(4999, 4)
(595, 5)
Bandwidth: 0.1, Kernel: gaussian, Accuracy: 0.28


C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Bandwidth: 0.1, Kernel: tophat, Accuracy: 0.23
Bandwidth: 0.1, Kernel: epanechnikov, Accuracy: 0.23
Bandwidth: 0.1, Kernel: exponential, Accuracy: 0.28
Bandwidth: 0.1, Kernel: linear, Accuracy: 0.23
Bandwidth: 0.1, Kernel: cosine, Accuracy: 0.23
Bandwidth: 0.2, Kernel: gaussian, Accuracy: 0.28
Bandwidth: 0.2, Kernel: tophat, Accuracy: 0.23
Bandwidth: 0.2, Kernel: epanechnikov, Accuracy: 0.23
Bandwidth: 0.2, Kernel: exponential, Accuracy: 0.28
Bandwidth: 0.2, Kernel: linear, Accuracy: 0.23
Bandwidth: 0.2, Kernel: cosine, Accuracy: 0.23
Bandwidth: 0.3, Kernel: gaussian, Accuracy: 0.28
Bandwidth: 0.3, Kernel: tophat, Accuracy: 0.23
Bandwidth: 0.3, Kernel: epanechnikov, Accuracy: 0.23


C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Bandwidth: 0.3, Kernel: exponential, Accuracy: 0.28
Bandwidth: 0.3, Kernel: linear, Accuracy: 0.23
Bandwidth: 0.3, Kernel: cosine, Accuracy: 0.23
Bandwidth: 0.4, Kernel: gaussian, Accuracy: 0.28
Bandwidth: 0.4, Kernel: tophat, Accuracy: 0.23
Bandwidth: 0.4, Kernel: epanechnikov, Accuracy: 0.23


...

C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Bandwidth: 2.0, Kernel: gaussian, Accuracy: 0.28
Bandwidth: 2.0, Kernel: tophat, Accuracy: 0.23
Bandwidth: 2.0, Kernel: epanechnikov, Accuracy: 0.23


C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Bandwidth: 2.0, Kernel: exponential, Accuracy: 0.28
Bandwidth: 2.0, Kernel: linear, Accuracy: 0.23
Bandwidth: 2.0, Kernel: cosine, Accuracy: 0.23
Best Accuracy: 0.28 with Bandwidth: 0.1 and Kernel: gaussian


C:\Python310\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
